## Advanced RAG - Retrieval Strategies
### Dense, Sparse, Hybrid, and Reranking

**Learning Objectives:**
- Understand different retrieval modes
- Implement hybrid search with filters
- Apply reranking for better results
- Build reusable retrieval functions

![image-3.png](attachment:image-3.png)

In [ ]:
!pip install -U langchain langchain-community langchain-google-genai sentence-transformers qdrant-client langchain-qdrant fastembed fastembed-gpu

In [3]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore, RetrievalMode, FastEmbedSparse
from qdrant_client import models


# re-ranking for better result
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# metadata filtering
from qdrant_client.models import Filter, FieldCondition, MatchValue

# metadata extraction from LLM
from scripts.schema import ChunkMetadata

In [4]:
# Configuration
COLLECTION_NAME = "financial_docs"
LLM_MODEL = "gemini-2.5-flash"

RERANKER_MODEL = "BAAI/bge-reranker-base"

### Initialize LLM and Vector Store

In [5]:
from google.colab import userdata

In [ ]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(model=LLM_MODEL, api_key=userdata.get('GOOGLE_API_KEY'))



embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
    encode_kwargs={"normalize_embeddings": True}
)
# Gemini embeddings
# embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)

# Sparse embeddings
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

In [7]:
# Connect to existing collection
vector_store = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    collection_name=COLLECTION_NAME,
    url=userdata.get('QDRANT_URL'),
    api_key = userdata.get("QDRANT_API_KEY"),
    retrieval_mode=RetrievalMode.HYBRID
)

In [8]:
vector_store.client.get_collection(collection_name=COLLECTION_NAME)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, warnings=None, indexed_vectors_count=3048, points_count=3059, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors={'': VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'langchain-sparse': SparseVectorParams(index=None, modifier=None)}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optim

### Filter Extraction with LLM

In [9]:
def extract_filters(user_query: str):

    prompt = f"""
            Extract metadata filters from the query. Return None for fields not mentioned.

                <USER QUERY STARTS>
                {user_query}
                </USER QUERY ENDS>

                #### EXAMPLES
                COMPANY MAPPINGS:
                - Amazon/AMZN -> amazon
                - Google/Alphabet/GOOGL/GOOG -> google
                - Apple/AAPL -> apple
                - Microsoft/MSFT -> microsoft
                - Tesla/TSLA -> tesla
                - Nvidia/NVDA -> nvidia
                - Meta/Facebook/FB -> meta

                DOC TYPE:
                - Annual report -> 10-k
                - Quarterly report -> 10-q
                - Current report -> 8-k

                EXAMPLES:
                "Amazon Q3 2024 revenue" -> {{"company_name": "amazon", "doc_type": "10-q", "fiscal_year": 2024, "fiscal_quarter": "q3"}}
                "Apple 2023 annual report" -> {{"company_name": "apple", "doc_type": "10-k", "fiscal_year": 2023}}
                "Tesla profitability" -> {{"company_name": "tesla"}}

                Extract metadata based on the user query only:
            """

    structurerd_llm = llm.with_structured_output(ChunkMetadata)

    metadata = structurerd_llm.invoke(prompt)

    filters = metadata.model_dump(exclude_none=True)

    return filters


In [10]:
query = "what is amazon's revenue in 2023 in q1?"
filters = extract_filters(query)

In [11]:
filters

{'company_name': 'amazon',
 'doc_type': '10-q',
 'fiscal_year': '2023',
 'fiscal_quarter': 'q1'}

In [12]:
filters

{'company_name': 'amazon',
 'doc_type': '10-q',
 'fiscal_year': '2023',
 'fiscal_quarter': 'q1'}

## Added Payload Index for Filtering

In [13]:
client = vector_store.client

In [14]:
def ensure_indices(keys: list):
    current_indexes = client.get_collection(COLLECTION_NAME).payload_schema
    for key in keys:
        field_path = f"metadata.{key}"
        if field_path not in current_indexes:
            print(f"Creating index for {field_path}...")
            client.create_payload_index(
                collection_name=COLLECTION_NAME,
                field_name=field_path,
                field_schema=models.PayloadSchemaType.KEYWORD, # Default to keyword
            )

# Usage inside hybrid_search
filters = extract_filters(query)
if filters:
    ensure_indices(filters.keys())
    # ... rest of your code

### Retrieval Functions

In [15]:
# metadata filtering
from qdrant_client.models import Filter, FieldCondition, MatchValue


[FieldCondition(key=f"metadata.{key}", match=MatchValue(value=value))  for key, value in filters.items()]

[FieldCondition(key='metadata.company_name', match=MatchValue(value='amazon'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None),
 FieldCondition(key='metadata.doc_type', match=MatchValue(value='10-q'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None),
 FieldCondition(key='metadata.fiscal_year', match=MatchValue(value='2023'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None),
 FieldCondition(key='metadata.fiscal_quarter', match=MatchValue(value='q1'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None)]

In [16]:
def hybrid_search(query: str, k: int = 5):
    """
    Perform hybrid search (dense + sparse vectors).

    Args:
        query: Search query
        k: Number of results
        filters: Optional filters like {"company_name": "amazon", "fiscal_year": 2024}

    Returns:
        List of Document objects
    """

    filters = extract_filters(query)
    print(filters)
    qdrant_filter = None

    if filters:
        condition = [FieldCondition(key=f"metadata.{key}", match=MatchValue(value=value))
                     for key, value in filters.items()]

        qdrant_filter = Filter(must=condition)

    results = vector_store.similarity_search(query=query, k=k, filter=qdrant_filter)

    return results


In [17]:
query = "what is the tesla's revenue"

results = hybrid_search(query, k=10)

{'company_name': 'tesla'}


In [18]:
results

[]

In [19]:
query = "what is amazon's cashflow in 2024 in q1?"

results = hybrid_search(query, k=10)

{'company_name': 'amazon', 'doc_type': '10-q', 'fiscal_year': '2024', 'fiscal_quarter': 'q1'}


In [21]:
results

[Document(metadata={'company_name': 'amazon', 'doc_type': '10-q', 'fiscal_quarter': 'q1', 'fiscal_year': '2024', 'content_type': 'text', 'file_hash': '2b01409631890a7a74ee8c3ff49f0d313201aec0f24874a7f595e5a3f57c17eb', 'source_file': 'amazon 10-q q1 2024.md', 'page': 28, '_id': '70ac0c98-dff2-400f-bbe5-49d16d851ae8', '_collection_name': 'financial_docs'}, page_content="\n\ncosts, our level of productivity and accuracy, changes in volume, size, and weight of units received and fulfilled, the extent to which third-party sellers utilize Fulfillment by Amazon services, timing of fulfillment network and physical store expansion, the extent we utilize fulfillment services provided by third parties, mix of products and services sold, and our ability to affect customer service contacts per unit by implementing improvements in our operations and enhancements to our customer self-service features. Additionally, sales by our sellers have higher payment processing and related transaction costs as a

In [ ]:
# re-ranking for better result
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


def rerank_results(query: str, documents: list, top_k: int = 5):
    """
    Rerank documents using cross-encoder.

    Args:
        query: Search query
        documents: List of Document objects
        top_k: Number of top results to return

    Returns:
        List of (score, Document) tuples sorted by relevance
    """

    reranker = HuggingFaceCrossEncoder(model_name=RERANKER_MODEL, model_kwargs = {'device': 'cuda'})

    query_doc_pairs = [(query, doc.page_content) for doc in documents]

    scores = reranker.score(query_doc_pairs)

    reranked = sorted(zip(scores, documents), key=lambda x: x[0], reverse=True)

    reranked = reranked[:top_k]
    return [rank[1] for rank in reranked]


response = rerank_results(query, results)


In [23]:
response

[Document(metadata={'company_name': 'amazon', 'doc_type': '10-q', 'fiscal_quarter': 'q1', 'fiscal_year': '2024', 'content_type': 'text', 'file_hash': '2b01409631890a7a74ee8c3ff49f0d313201aec0f24874a7f595e5a3f57c17eb', 'source_file': 'amazon 10-q q1 2024.md', 'page': 26, '_id': 'e60a5c91-f752-4fbb-98f1-c911e435a470', '_collection_name': 'financial_docs'}, page_content='\n\nwere driven largely by our continued focus on price, selection, and convenience for our customers, including from our fast shipping offers. Changes in foreign exchange rates reduced International net sales by $248 million for Q1 2024.\n\nAWS sales increased 17% in Q1 2024 compared to the comparable prior year period. The sales growth primarily reflects increased customer usage, partially offset by pricing changes primarily driven by long-term customer contracts.\n\n## Operating Income (Loss)\n\nOperating income (loss) by segment is as follows (in millions):\n\n|                         | Three Months Ended March 31,  

### Example: Dynamic Filters

In [24]:
query = "what is the revenue of apple in 2024?"

results = hybrid_search(query, k=5)

{'company_name': 'apple', 'fiscal_year': '2024'}


In [25]:
results

[Document(metadata={'company_name': 'apple', 'doc_type': '10-q', 'fiscal_quarter': 'q1', 'fiscal_year': '2024', 'content_type': 'tables', 'file_hash': '835dfa890352f2cb416e48b27a695ffadaf71189c126061305eff6888dcc02ed', 'source_file': 'apple 10-q q1 2024', 'page': 9, '_id': 'e78de28b-9235-4c46-8254-91697f586e9c', '_collection_name': 'financial_docs'}, page_content='**Page:** 9\n\n## Note 2 - Revenue\nNet sales disaggregated by significant products and services for the three- and six-month periods ended March 30, 2024 and April 1, 2023 were as follows (in millions):\n\n|                                 | Three Months Ended   | Three Months Ended   | Six Months Ended   | Six Months Ended   |\n|---------------------------------|----------------------|----------------------|--------------------|--------------------|\n|                                 | March 30, 2024       | April 1, 2023        | March 30, 2024     | April 1, 2023      |\n| iPhone ®                        | $ 45,963       

In [26]:
query = "what is the revenue of apple in 2023?"

results = hybrid_search(query, k=5)

results

{'company_name': 'apple', 'fiscal_year': '2023'}


[Document(metadata={'company_name': 'apple', 'doc_type': '8-k', 'fiscal_quarter': 'q4', 'fiscal_year': '2023', 'content_type': 'text', 'file_hash': '95e5b6266ef3d325d21625559d87201c5f08d87c8f08130b7f757145d3897d7e', 'source_file': 'apple 8-k q4 2023.md', 'page': 5, '_id': 'e1e690e7-31dc-4504-8d8d-d16616976d68', '_collection_name': 'financial_docs'}, page_content="\n\n## Apple reports first quarter results\n\n## Services revenue reaches new all-time record\n\n## EPS up 16 percent to new all-time high\n\nCUPERTINO, CALIFORNIA - Apple  today announced financial results for its fiscal 2024 first quarter ended December 30, 2023. The Company posted quarterly revenue of $119.6 billion, up 2 percent year over year, and quarterly earnings per diluted share of $2.18, up 16 percent year over year. ®\n\n'Today Apple is reporting revenue growth for the December quarter fueled by iPhone sales, and an all-time revenue record in Services,' said Tim Cook, Apple's CEO. 'We are pleased to announce that o